# 3.0.2 QA Agent – L04-01-Grübler (GPT-4o)

Alle erarbeiteten Ergebnisse aus Kapitel 2 und 3 sind mögliche Vorverarbeitungsschritte. Diese Informationen können z.B. via QA-Prompts zugänglich gemacht werden. RAG ermöglicht domänenspezifische Aufgaben zu lösen, ganz ohne Fine-Tuning.

* Bebauungsplan
* BauNVO

Ziel: Tabelle ausfüllen via KI-System (QA).

In [11]:
from utils.openai import OpenAI
from utils.pprint import pprint

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
ava = OpenAI(instructions)

# Kein CopyPaste => StoreMagic
%store instructions
%store -r msg102
%store -d hello_world
%store

Stored 'instructions' (str)
Stored variables and their in-db values:
instructions             -> 'Du bist ein Assistent zur getreuen Wiedergabe von
msg102                   -> '```json\n{"Gültig seit": 2000}\n```'


In [ ]:
# * GPT-4o ist in der Lage Informationen zu extrahieren (Zero-Shot)
# * ZIEL: Wie Daten miteinander vorverarbeiten + verbinden, sodass QA bzgl. Bebauungsplan funktioniert.

# 1. OCR-Agent (Pre-Processing-Pipeline)
# e.g. 'Du bist ein OCR-Programm, was alle textlichen Informationen aus einem Bild extrahiert'

# 2. QA-Agent (Chat-Bot)
# e.g. 'Du bist ein Chat-Bot, der mithilfe der gegebenen Informationen, Fragen beantwortet. Helfen die gegeben Informationen nicht um eine Antwort zu formulieren, antworte mit "Das ist mir nicht bekannt"'

In [ ]:
ze = '''
### Art der baulichen Nutzung
- **Gewerbegebiete (GE)** (§ 8 BauNVO)
- **Gewerbegebiete eingeschränkt (GEE)** (§ 8 BauNVO)

### Maß der baulichen Nutzung
- **Gebäudehöhe (GH)**: max. 12,00 m
- **Grundflächenzahl (GRZ)** (§ 16 BauNVO): 0,6
- **Geschossflächenzahl (GFZ)** (§ 16 BauNVO): 1,2

### Bauweise
- **Abweichende Bauweise** (§ 22 BauNVO)

### Überbaubare Grundstücksfläche
- **Baugrenze** (§ 23 (3) BauNVO)
- **Umgrenzung der Flächen, die von der Bebauung freizuhalten sind** (§ 9 (1) 10 BauGB)
'''

pz = '''
[
  {
    "Art der Nutzung": "GEE*",
    "Gebäudehöhe": "max. 12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "a"
  },
  {
    "Art der Nutzung": "GEE",
    "Gebäudehöhe": "max. 12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "a"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max. 12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "a"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max. 12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "a"
  }
]
'''

tt= '''
### Art der baulichen Nutzung:
1.1 Art der baulichen Nutzung (§§ 1 - 15 BauNVO)
1.1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)
1.1.1.1 Zulässig sind / zulässig ist:
1.1.1.1.1 Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
1.1.1.1.2 Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
1.1.1.2 Nicht zulässig sind / nicht zulässig ist:
1.1.1.2.1 Nutzungen gemäß § 8 (3) Nr.3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
1.1.1.2.2 Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemäß § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO nicht zulässig.
1.1.2 Gewerbegebiet (GE) (§ 8 BauNVO)
1.1.2.1 Zulässig sind / zulässig ist:
1.1.2.1.1 Gewerbebetriebe aller Art, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
1.1.2.1.2 Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
1.1.2.2 Nicht zulässig sind / nicht zulässig ist:
1.1.2.2.1 Nutzungen gemäß § 8 (3) Nr.3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
1.1.2.2.2 Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemäß § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO nicht zulässig.

### Maß der baulichen Nutzung:
1.2 Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)
1.2.1 Grundflächenzahl (§ 19 BauNVO) - siehe Einschreibung im Lageplan
1.2.2 Geschossflächenzahl (§ 20 BauNVO) - siehe Einschreibung im Lageplan
1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO) - siehe Einschreibung im Lageplan
Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.

### Bauweise:
1.3 Bauweise (§ 22 BauNVO) - siehe Einschreibung im Lageplan
a) abweichende Bauweise. Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.

### Überbaubare Grundstücksfläche:
1.4 Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)
Die überbaubare Grundstücksfläche wird durch die Festsetzung von Baugrenzen in der Planzeichnung bestimmt.
Garagen im Sinne von § 12 BauNVO und Nebenanlagen im Sinne des § 14 BauNVO sowie Anlagen der Fremdwerbung und bauliche Anlagen, die nach Landesrecht in den Abstandsflächen zulässig sind oder zugelassen werden können, sind nur innerhalb der überbaubaren Grundstücksflächen zulässig.

### Referenzen:
Seitenzahlen: 2, 3, 4
'''

ava.addContext([
  { 
    "type": "text",
    "text": f'Informationen aus dem zeichnerischen Teil (Planzeichenerklärung): {ze}'
  },
  { 
    "type": "text",
    "text": f'Informationen aus dem zeichnerischen Teil (Planzeichnung): {pz}'
  },
  { 
    "type": "text",
    "text": f'Informationen aus dem schriftlichen Teil: {tt}'
  },
])

msg = ava.request([
  {
          "type": "text",
          "text": 'Setze die gesammelten Informationen aus dem schriftlichen und zeichnerischen Teil des Bebauungsplans in Beziehung. Output UTF-8 encoded.'
      },
  ]),

pprint(msg)
ava.clearContext()